# Part 1

#### Completing `bust_probability` function

In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.optim as optim
import sklearn
from sklearn import metrics as metrics
import pandas as pd
import math

In [2]:
def data_loader(path, table_idx, player_or_dealer):
    #utility for loading train.csv, example use in the notebook
    data = pd.read_csv(path, header=[0,1,2])
    spy = data[(f'table_{table_idx}', player_or_dealer, 'spy')]
    card = data[(f'table_{table_idx}', player_or_dealer, 'card')]
    return np.array([spy, card]).T

In [3]:
def bust_probability(dealer_cards):
    """
    dealer_cards: list -> integer series of player denoting value of cards observed upto this point

    Current body is random for now, change it accordingly
    
    output: probability of going bust on this table
    """
    bust_count = 0
    total_cards = len(dealer_cards)
    score = 0
    for card in dealer_cards:
        score += card
        if score > 16:
            if score > 21:
                bust_count += 1
            score = 0
    return bust_count/total_cards

In [4]:
for table_idx in range(0, 5):
    dealer_data = data_loader("data/train.csv", table_idx, "dealer")
    dealer_cards = dealer_data[:,1]
    print(table_idx, bust_probability(dealer_cards))

0 0.18865
1 0.10155
2 0.09175
3 0.07995
4 0.00055


### Two additional points for "willingness to play"

### **1. Effective House Edge (EHE)**  
#### **Definition:**  
This metric estimates the actual house edge at a given table based on observed outcomes rather than theoretical expectations. It measures the average percentage of a player’s bet lost per round.  

#### **Formula:**  
$$
EHE = 1 - \frac{\text{Total player winnings}}{\text{Total player bets}}
$$
where:  
- **Total player winnings** = sum of all amounts won by players.  
- **Total player bets** = sum of all bets placed by players.  

#### **Strengths:**  
- Captures the real-world profitability of a table.  
- Accounts for both dealer performance and player behavior.  

#### **Weaknesses:**  
- Requires tracking actual bet sizes and payouts.  
- Can be skewed if a few players make bad decisions.  

---

### **2. Volatility Score (VS)**  
#### **Definition:**  
Measures how unpredictable the dealer’s bust rate is across different hands.  
A high volatility score suggests that the dealer’s performance is inconsistent.

#### **Formula:**  
$$
VS = \sigma_b = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (B_i - \mu_b)^2}
$$
where:  
- \( $B_i$ \) is the bust probability observed in past hands.  
- \( $\mu_b$ \) is the mean bust probability.  
- \( $\sigma_b$ \) is the standard deviation of bust probabilities.

#### **Strengths:**  
- Identifies tables with consistent dealer behavior (lower volatility is preferable).  
- Helps risk-averse players avoid unpredictable situations.

#### **Weaknesses:**  
- Doesn’t directly indicate profitability.  
- Requires enough historical data for accurate estimation.

# Part 2

In [ ]:
def get_card_value_from_spy_value(value : float) -> int:
    """
    Implement here. Please make sure that the output of this function is an integer.
    """
    value += 100
    value += 2.5
    value = math.trunc(value)
    value = value % 10
    if value <= 1:
        value += 10
    return value

## Motivation
We observed the card values for various spy values for each of the table indices. We divided the array based on the card values, and saw the spy values for each card index (we also sorted the spy values). From this, we were able to observe the pattern of jump of 10 for each card index, and also the bound of 0.5 range. 


# Part 3

In [9]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn import metrics as metrics
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import pickle
import os

def data_loader(path, table_idx, player_or_dealer):
    """Loads the data for that table index."""
    data = pd.read_csv(path, header=[0,1,2])
    spy = data[(f'table_{table_idx}', player_or_dealer, 'spy')]
    card = data[(f'table_{table_idx}', player_or_dealer, 'card')]
    return np.array([spy, card]).T

class SimpleNN(nn.Module):
    """Simple NN Architecture"""
    def __init__(self, input_size, hidden_size=16):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
    
    def forward(self, x):
        return self.model(x)

class MyPlayer:
    def __init__(self, table_index):
        self.table_index = table_index
        self.player_model = None
        self.dealer_model = None
        self.player_model_type = None
        self.dealer_model_type = None
        self.player_scaler = None
        self.dealer_scaler = None
        self._load_or_train_models()
    
    def _load_or_train_models(self):
        """Load pre-trained models or train new ones"""
        train_file = 'train.csv' 
        player_model_path = f'player_model_table_{self.table_index}.pkl'
        dealer_model_path = f'dealer_model_table_{self.table_index}.pkl'
        player_scaler_path = f'player_scaler_table_{self.table_index}.pkl'
        dealer_scaler_path = f'dealer_scaler_table_{self.table_index}.pkl'
        player_type_path = f'player_type_table_{self.table_index}.txt'
        dealer_type_path = f'dealer_type_table_{self.table_index}.txt'
        if os.path.exists(player_model_path) and os.path.exists(dealer_model_path):
            with open(player_model_path, 'rb') as f:
                self.player_model = pickle.load(f)
            with open(player_type_path, 'r') as f:
                self.player_model_type = f.read().strip()
            if os.path.exists(player_scaler_path):
                with open(player_scaler_path, 'rb') as f:
                    self.player_scaler = pickle.load(f)
            with open(dealer_model_path, 'rb') as f:
                self.dealer_model = pickle.load(f)
            with open(dealer_type_path, 'r') as f:
                self.dealer_model_type = f.read().strip()
            if os.path.exists(dealer_scaler_path):
                with open(dealer_scaler_path, 'rb') as f:
                    self.dealer_scaler = pickle.load(f)
        else:
            if os.path.exists(train_file):
                self._train_models(train_file)
            else:
                self.player_model_type = 'linear'
                self.dealer_model_type = 'linear'
                self.player_model = LinearRegression()
                self.dealer_model = LinearRegression()
    
    def _train_models(self, train_file):
        """Train models using data from train.csv"""
        try:
            data = pd.read_csv(train_file, header=[0, 1, 2])
            player_spy = data[(f'table_{self.table_index}', 'player', 'spy')].values
            dealer_spy = data[(f'table_{self.table_index}', 'dealer', 'spy')].values
            X_player, y_player = self._create_sequence_data(player_spy)
            X_dealer, y_dealer = self._create_sequence_data(dealer_spy)
            player_model_info = self._select_and_train_best_model(X_player, y_player, 'player')
            dealer_model_info = self._select_and_train_best_model(X_dealer, y_dealer, 'dealer')
            self.player_model = player_model_info['model']
            self.player_model_type = player_model_info['type']
            self.player_scaler = player_model_info.get('scaler')
            self.dealer_model = dealer_model_info['model']
            self.dealer_model_type = dealer_model_info['type']
            self.dealer_scaler = dealer_model_info.get('scaler')
            self._save_models()
        except Exception as e:
            print(f"Error during training: {e}")
            self.player_model_type = 'linear'
            self.dealer_model_type = 'linear'
            self.player_model = LinearRegression()
            self.dealer_model = LinearRegression()
    
    def _create_sequence_data(self, series, lag=5):
        """Create sequence data with lag features for time series prediction"""
        X, y = [], []
        for i in range(lag, len(series)):
            X.append(series[i-lag:i])
            y.append(series[i])
        return np.array(X), np.array(y)
    
    def _select_and_train_best_model(self, X, y, role):
        """Select and train the best model for the data based on simple validation"""
        if len(X) < 20:
            model = LinearRegression()
            model.fit(X, y)
            return {'model': model, 'type': 'linear'}
        split = int(0.8 * len(X))
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]
        models = {
            'linear': LinearRegression(),
            'forest': RandomForestRegressor(n_estimators=100, random_state=42),
            'nn': None
        }
        
        best_mse = float('inf')
        best_model_info = None
        for model_type, model in models.items():
            if model_type == 'nn':
                continue
                
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            mse = np.mean((y_val - y_pred) ** 2)
            
            if mse < best_mse:
                best_mse = mse
                best_model_info = {'model': model, 'type': model_type}
        if len(X_train) > 50:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            X_train_tensor = torch.FloatTensor(X_train_scaled)
            y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1))
            X_val_tensor = torch.FloatTensor(X_val_scaled)
            model = SimpleNN(X_train.shape[1])
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            epochs = 200
            for epoch in range(epochs):
                outputs = model(X_train_tensor)
                loss = criterion(outputs, y_train_tensor)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            model.eval()
            with torch.no_grad():
                y_pred = model(X_val_tensor).numpy().flatten()
            mse = np.mean((y_val - y_pred) ** 2)
            if mse < best_mse:
                best_mse = mse
                best_model_info = {'model': model, 'type': 'nn', 'scaler': scaler}
        if best_model_info['type'] == 'nn':
            X_scaled = best_model_info['scaler'].fit_transform(X)
            X_tensor = torch.FloatTensor(X_scaled)
            y_tensor = torch.FloatTensor(y.reshape(-1, 1))
            model = SimpleNN(X.shape[1])
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            epochs = 300
            for epoch in range(epochs):
                outputs = model(X_tensor)
                loss = criterion(outputs, y_tensor)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            best_model_info['model'] = model
        else:
            best_model_info['model'].fit(X, y)
        
        return best_model_info
    
    def _save_models(self):
        """Save trained models to disk"""
        with open(f'player_model_table_{self.table_index}.pkl', 'wb') as f:
            pickle.dump(self.player_model, f)
        with open(f'player_type_table_{self.table_index}.txt', 'w') as f:
            f.write(self.player_model_type)
        if self.player_scaler is not None:
            with open(f'player_scaler_table_{self.table_index}.pkl', 'wb') as f:
                pickle.dump(self.player_scaler, f)
        with open(f'dealer_model_table_{self.table_index}.pkl', 'wb') as f:
            pickle.dump(self.dealer_model, f)
        with open(f'dealer_type_table_{self.table_index}.txt', 'w') as f:
            f.write(self.dealer_model_type)
        if self.dealer_scaler is not None:
            with open(f'dealer_scaler_table_{self.table_index}.pkl', 'wb') as f:
                pickle.dump(self.dealer_scaler, f)
    
    def get_card_value_from_spy_value(self, value):
        """
        value: a value from the spy series as a float
        Output: return a scalar value of the prediction
        """
        value += 100
        value += 2.5
        value = math.trunc(value)
        value = value % 10
        if value <= 1:
            value += 10
        return value
    
    def get_player_spy_prediction(self, hist):
        """
        hist: a 1D numpy array of size (len_history,) len_history=5
        Output: return a scalar value of the prediction
        """
        X = hist.reshape(1, -1)
        if self.player_model_type == 'nn':
            X_scaled = self.player_scaler.transform(X)
            X_tensor = torch.FloatTensor(X_scaled)
            self.player_model.eval()
            with torch.no_grad():
                prediction = self.player_model(X_tensor).item()
        else:
            prediction = self.player_model.predict(X)[0]
        
        return prediction
    
    def get_dealer_spy_prediction(self, hist):
        """
        hist: a 1D numpy array of size (len_history,) len_history=5
        Output: return a scalar value of the prediction
        """
        X = hist.reshape(1, -1)
        if self.dealer_model_type == 'nn':
            X_scaled = self.dealer_scaler.transform(X)
            X_tensor = torch.FloatTensor(X_scaled)
            self.dealer_model.eval()
            with torch.no_grad():
                prediction = self.dealer_model(X_tensor).item()
        else:
            prediction = self.dealer_model.predict(X)[0] 
        return prediction